<a href="https://colab.research.google.com/github/SuiLing237/Google-Ignite-Tealth/blob/main/Copy_of_Tealth_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Disclaimer
* Based on Google Tensorflow Tutorial as the Base Code
* Edited by Vincent Tatan for Intro to CNN Lessons

# Base Code
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

# Image classification and Building Simple CNN

This tutorial shows how to classify cats or dogs from images. It builds an image classifier using a `tf.keras.Sequential` model and load data using `tf.keras.preprocessing.image.ImageDataGenerator`. You will get some practical experience and develop intuition for the following concepts:

* Building _data input pipelines_ using the `tf.keras.preprocessing.image.ImageDataGenerator` class to efficiently work with data on disk to use with the model.
* _Overfitting_ —How to identify and prevent it.
* _Data augmentation_ and _dropout_ —Key techniques to fight overfitting in computer vision tasks to incorporate into the data pipeline and image classifier model.

This tutorial follows a basic machine learning workflow:

1. Examine and understand data
2. Build an input pipeline
3. Build the model
4. Train the model
5. Test the model
6. Improve the model and repeat the process

## Import Tensorflow packages

Let's start by importing the required packages. The `os` package is used to read files and directory structure, NumPy is used to convert python list to numpy array and to perform required matrix operations and `matplotlib.pyplot` to plot the graph and display images in the training and validation data.

At the time of writing, Colab has TensorFlow version 1.x installed by default. TensorFlow 2.x is much easier to use, so let's start with that. To switch to 2.x we'll use the magic command below. Note, you can also install TensorFlow by using pip, but in Colab, the magic command is faster.

Import Tensorflow and the Keras classes needed to construct our model.

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

In [ ]:
# We'll use Keras as the TensorFlow's user-friendly API to deep learning

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

## Load data

Begin by downloading the dataset. This tutorial uses a filtered version of <a href="https://www.kaggle.com/c/dogs-vs-cats/data" target="_blank">Dogs vs Cats</a> dataset from Kaggle. Download the archive version of the dataset and store it in the "/tmp/" directory.

In [ ]:
! kaggle datasets download -d pushkar34/teeth-dataset

In [ ]:
# importing required modules
from zipfile import ZipFile
  
# specifying the zip file name
file_name = "teeth-dataset.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as z:
    # printing all the contents of the zip file
    z.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    results = z.extractall()
    print('Done!')

In [ ]:
PATH = os.path.join(os.path.dirname('/content/teeth_dataset/teeth_dataset'), 'teeth_dataset')
print(PATH)

The dataset has the following directory structure:

<pre>
<b>cats_and_dogs_filtered</b>
|__ <b>train</b>
    |______ <b>cats</b>: [cat.0.jpg, cat.1.jpg, cat.2.jpg ....]
    |______ <b>dogs</b>: [dog.0.jpg, dog.1.jpg, dog.2.jpg ...]
|__ <b>validation</b>
    |______ <b>cats</b>: [cat.2000.jpg, cat.2001.jpg, cat.2002.jpg ....]
    |______ <b>dogs</b>: [dog.2000.jpg, dog.2001.jpg, dog.2002.jpg ...]
</pre>

After extracting its contents, assign variables with the proper file path for the training and validation set.

In [ ]:
train_dir = os.path.join(PATH, 'Trianing')
validation_dir = os.path.join(PATH, 'test')

In [ ]:
train_bad_dir = os.path.join(train_dir, 'caries')  # directory with our training bad pictures
train_good_dir = os.path.join(train_dir, 'without_caries')  # directory with our training good pictures
validation_bad_dir = os.path.join(validation_dir, 'caries')  # directory with our validation bad pictures
validation_good_dir = os.path.join(validation_dir, 'no-caries')  # directory with our validation good pictures

In [ ]:
print(os.listdir(train_bad_dir))

### Understand the data

Let's look at how many cats and dogs images are in the training and validation directory:

In [ ]:
num_bad_tr = len(os.listdir(train_bad_dir))
num_good_tr = len(os.listdir(train_good_dir))

num_bad_val = len(os.listdir(validation_bad_dir))
num_good_val = len(os.listdir(validation_good_dir))

total_train = num_bad_tr + num_good_tr
total_val = num_bad_val + num_good_val

In [ ]:
print('total training bad images:', num_bad_tr)
print('total training good images:', num_good_tr)

print('total validation bad images:', num_bad_val)
print('total validation good images:', num_good_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

For convenience, set up variables to use while pre-processing the dataset and training the network.

In [ ]:
batch_size = 5
epochs = 30
IMG_HEIGHT = 150
IMG_WIDTH = 150

## Data preparation

Format the images into appropriately pre-processed floating point tensors before feeding to the network:

1. Read images from the disk.
2. Decode contents of these images and convert it into proper grid format as per their RGB content.
3. Convert them into floating point tensors.
4. Rescale the tensors from values between 0 and 255 to values between 0 and 1, as neural networks prefer to deal with small input values. It's important that the training set and the testing set are preprocessed in the same way.

Fortunately, all these tasks can be done with the `ImageDataGenerator` class provided by `tf.keras`. It can read images from disk and preprocess them into proper tensors. It will also set up generators that convert these images into batches of tensors—helpful when training the network.

In [ ]:
train_image_generator = ImageDataGenerator(rotation_range=30,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              rescale=1/255,
                              shear_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip=True,
                              fill_mode='nearest') # Generator for our training data
validation_image_generator = ImageDataGenerator(rotation_range=30,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              rescale=1/255,
                              shear_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip=True,
                              fill_mode='nearest') # Generator for our validation data

After defining the generators for training and validation images, the `flow_from_directory` method load images from the disk, applies rescaling, and resizes the images into the required dimensions.

In [ ]:
image_shape=(150, 150, 3)

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           target_size=image_shape[:2],
                                                           class_mode='binary')

In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=image_shape[:2],
                                                              class_mode='binary')

### Visualize training images

Visualize the training images by extracting a batch of images from the training generator—which are 128 images in this example—then plot five of them with `matplotlib`.

In [ ]:
sample_training_images, _ = next(train_data_gen)

The `next` function returns a batch from the dataset. The return value of `next` function is in form of `(x_train, y_train)` where x_train is training features and y_train, its labels. Discard the labels to only visualize the training images.

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_training_images[:5])

## Create the model

### What does a layer do?

Layers extract data representations fed to them
1. First layer: receives pixel values as input. Learning to detect edges as combinations of pixel
2. Second layer: Receives edges as inputs and detect lines
3. n layer after: detect shapes and high level features.

Deep learning refers to this depth which consist of chaining together dense layers. [tf.keras.layers.Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), have parameters that are initialized randomly, then tuned (or learned) during training by gradient descent.

The model consists of three convolution blocks with a max pool layer in each of them. There's a fully connected layer with 512 units on top of it that is activated by a `relu` activation function. Neural networks are made up of layers. Here, you'll define the layers, and assemble them into a model. We will start with a single Dense layer. 


In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

In [ ]:
model = Sequential()


model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(150, 150, 3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=(150, 150, 3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=(150, 150, 3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

[tf.keras.layers.Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), flattens 2-D Array into 1-D Array. 

Think of this layer as unstacking rows of pixels in the image and lining them up. This layer has no parameters to learn; it only reformats the data. This is necessary since Dense layers require 1-D array as input.

After the pixels are flattened, this model consists of a single Dense layer. This is a densely connected, or fully connected (FCL). 

The Dense last layer has 1 neurons with sigmoid activation 

After classifying an image, each neuron will contains a score that indicates the probability that the current image belongs to one of the 10 classes.

### Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model's compile step:

*Loss function* — This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction.

*Optimizer* — This is how the model is updated based on the data it sees and its loss function.

*Metrics* — Used to monitor the training and testing steps. The following example uses accuracy, the fraction of the images that are correctly classified.

For this tutorial we choose *ADAM* optimizer and *binary cross entropy* loss function. To view training and validation accuracy for each training epoch, pass the `metrics` argument.

In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer='Adam',
             metrics=['accuracy'])

### Model summary

View all the layers of the network using the model's `summary` method:

In [ ]:
model.summary()

### Train the model
Training the neural network model requires the following steps:

1. Feed the training data to the model. In this example, the training data is in the ```train_images``` and ```train_labels``` arrays.

1. The model learns to associate images and labels.

1. You ask the model to make predictions about a test set—in this example, the ```test_images``` array.

1. Verify that the predictions match the labels from the ```test_labels``` array.

To begin training, call the ```model.fit``` method — so called because it "fits" the model to the training data:

In [ ]:
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

In [ ]:
history_dict = history.history
print(history_dict.keys())

### Visualize training results

Now visualize the results after training the network.

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(24, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label='Training Accuracy')
plt.plot(epochs_range, val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

As you can see from the plots, training accuracy and validation accuracy are off by large margin and the model has achieved only around **70%** accuracy on the validation set.

This gap between training accuracy and test accuracy represents overfitting. Overfitting is when a machine learning model performs worse on new, previously unseen inputs than on the training data. 

An overfitted model "memorizes" the training data—with less accuracy on testing data.

Let's look at what went wrong and try to increase overall performance of the model.

In [ ]:
!pip install image

import matplotlib
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import cv2
#import matplotlib.image as mpimg
#from PIL import Image as pil_image
import image as pil_image
import matplotlib.pyplot as plt
%matplotlib inline
#get_ipython().run_line_magic('matplotlib', 'inline')
from keras import backend as K
import tensorflow as tf
from tkinter import *
from tkinter import ttk
from tkinter import filedialog

In [ ]:
matplotlib.use('Agg')

In [ ]:
actions = np.array(['bad', 'good'])

In [ ]:
PATH = os.path.join(os.path.dirname('/content/teeth_dataset/teeth_dataset/test/caries/wc1.jpg'), 'wc1.jpg')

img = tf.keras.utils.load_img(
    PATH, target_size=(IMG_HEIGHT, IMG_WIDTH)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
print(predictions)
score = tf.nn.softmax(predictions[0])
print(score)

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(actions[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
PATH = os.path.join(os.path.dirname('/content/teeth_dataset/teeth_dataset/test/no-caries/wc1.jpg'), 'nc1.jpg')

img = tf.keras.utils.load_img(
    PATH, target_size=(IMG_HEIGHT, IMG_WIDTH)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
print(predictions)
score = tf.nn.softmax(predictions[0])
print(score)

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(actions[np.argmax(score)], 100 * np.max(score))
)